<a href="https://colab.research.google.com/github/NiharikaSangitha/weather-app/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files

# Upload the RoomPlan.pdf and Schedule.pdf files
uploaded = files.upload()



In [5]:
!pip install opencv-python-headless pdf2image pillow numpy

In [8]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [11]:
from pdf2image import convert_from_path

# Convert PDF pages to images
def convert_pdf_to_images(pdf_path):
    return convert_from_path(pdf_path)

# Convert the Room Plan and Schedule PDFs
room_plan_images = convert_pdf_to_images('/content/RoomPlan.pdf')
schedule_images = convert_pdf_to_images('/content/Schedule.pdf')

# Display the first page of the Room Plan
room_plan_images[0].save('room_plan_page_0.png', 'PNG')
schedule_images[0].save('schedule_page_0.png', 'PNG')

In [13]:
# Function to match templates
def match_template(room_img, template_img):
    room_gray = cv2.cvtColor(room_img, cv2.COLOR_BGR2GRAY)
    template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)

    # Template Matching using OpenCV
    result = cv2.matchTemplate(room_gray, template_gray, cv2.TM_CCOEFF_NORMED)
    threshold = 0.8  # Set a threshold for matching
    locations = np.where(result >= threshold)

    # Count the number of matches
    count = len(list(zip(*locations[::-1])))
    return count

# Load images from saved files
room_image = cv2.imread('room_plan_page_0.png')
schedule_image = cv2.imread('schedule_page_0.png')

# Perform template matching and count occurrences
count = match_template(room_image, schedule_image)
print(f"Count of matched templates: {count}")


Count of matched templates: 0


In [14]:
icon_counts = {}
for i, schedule_img in enumerate(schedule_images):
    count = match_template(room_image, np.array(schedule_img))
    icon_counts[f'Icon_{i}'] = count

print("Icon Counts:", icon_counts)

# Function to generate a CSV report
import csv

def generate_report(icon_counts, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['Component', 'Count']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for icon, count in icon_counts.items():
            writer.writerow({'Component': icon, 'Count': count})

# Generate and download the CSV report
generate_report(icon_counts, 'output_report.csv')

from google.colab import files
files.download('output_report.csv')

Icon Counts: {'Icon_0': 0}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
import zipfile

# Create a ZIP file with the report and any necessary files
with zipfile.ZipFile("Assessment_Niharika.zip", 'w') as zipf:
    zipf.write('output_report.csv')

# Download the ZIP file
files.download('Assessment_Niharika.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>